# **TIỀN XỬ LÝ** 

## Các bước thực hiện:
 - Kiểm tra và tiền xử lý output 
 - Tách dữ liệu thành 3 tập: train, validaton, test.
 - Tiến hành tiền xử lý


In [1]:
# IMPORT THƯ VIỆN
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

## Khám phá dữ liệu (đủ để có thể xác định câu hỏi)

In [2]:
data = pd.read_csv('HOUSEs_info.csv', sep = '\t')
data

,Loại hình,Phòng ngủ,Phòng tắm,Diện tích,Diện tích sử dụng,Tình hình nội thất,Loại chủ quyền,Dự án,Tháp,Giá bán,...,Q2 Thảo Điền,Căn hộ cao cấp quận 2,Chủ đầu tư Singapore Thảo Điền,Căn hộ Thảo Điền,Căn hộ hạng sang,Căn hộ view sông,Đầy đủ vệ sinh,Kệ bếp,Hoàn thiện cơ bản,Nhà trống
0,Căn hộ,3.0,2.0,88 m²,88 m²,Đầy đủ nội thất,Sổ hồng,Sky Garden,Sky Garden 2,3 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Căn hộ,3.0,2.0,90 m²,86 m²,Đầy đủ nội thất,HĐ mua bán,The Sun Avenue,Block 2,4.9 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Căn hộ,4.0,2.0,128 m²,128 m²,Nội thất cơ bản,Sổ hồng,NaN,NaN,4.25 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nhà phố,4.0,4.0,31.8 m²,76.3 m²,Không có nội thất,Sổ hồng,NaN,NaN,5.62 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nhà phố,2.0,1.0,26.5 m²,54 m²,Không có nội thất,Sổ hồng,NaN,NaN,2.4 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6930,Căn hộ,2.0,1.0,69 m²,63 m²,Nội thất cơ bản,HĐ mua bán,Vinhomes Grand Park,S3.03,2.6 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6931,Căn hộ,2.0,2.0,65.4 m²,60.7 m²,Nội thất cơ bản,HĐ mua bán,Lovera Vista Khang Điền,BLOCK C,2.05 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6932,Căn hộ,2.0,2.0,66.4 m²,66.4 m²,Đầy đủ nội thất,HĐ mua bán,Masteri Thảo Điền,T1,8 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6933,Căn hộ,2.0,2.0,67 m²,67 m²,Nội thất cơ bản,NaN,Green Star Sky Garden,NaN,2.3 tỷ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [3]:
data.shape


(6935, 123)

### Dữ liệu có các dòng bị lặp không?

In [4]:
data.index.duplicated().sum()

0

## Đưa ra câu hỏi cần trả lời
*Output - Giá bán -* được tính từ *input - các thông tin của hành khách -* theo công thức nào?
 Trong đồ án này, ý nghĩa thực tế của việc đi tìm câu trả lời cho câu hỏi trên giúp người dùng có được 1 sự tham khảo với 1 độ chính xác cụ thể về giá nhà dựa trên các thông tin. Qua đó, người dùng có thể dựa trên dự đoán để tự đưa ra quyết định về việc mua (bán) nhà với một mức giá hợp lý.  


## Khám phá dữ liệu 
### 1. OUTPUT TRƯỚC KHI TIỀN XỬ LÝ

In [5]:
# Cột output hiện có kiểu dữ liệu gì?
data['Giá bán'].dtype

dtype('O')

In [6]:
# Cột output có giá trị thiếu không?
data['Giá bán'].isna().sum()

393

#### TIỀN XỬ LÝ OUTPUT

In [7]:
# Xoá ouput có giá trị thiếu
data = data[data['Giá bán'].notna()]
data['Giá bán'].isna().sum()
data = data.reset_index(drop = True) 

In [8]:
# Chuẩn hoá về dạng số

# Vị trí các số hảng tỷ
mil_index = [] 
# Vị trí các số hảng triệu
bil_index = []

for i in range(0, data.shape[0]):
    value = data['Giá bán'][i] 
    if 'tỷ' in value:
        data.at[i, 'Giá bán'] = value.rstrip('tỷ')
        bil_index.append(i)
    elif 'tr' in value:
        data.at[i, 'Giá bán'] = value.rstrip(' tr')
        mil_index.append(i)

In [9]:
# Chuyển đơn vị triệu thành tỷ (đồng)
data['Giá bán'] = data['Giá bán'].astype(float)
for i in mil_index:
    data.at[i, 'Giá bán'] = data['Giá bán'][i] / 1000

### 2. SAU KHI TIỀN XỬ LÝ OUTPUT

In [10]:
# Cột output hiện có kiểu dữ liệu gì?
data['Giá bán'].dtype

dtype('float64')

In [11]:
# Cột output có giá trị thiếu không?
data['Giá bán'].isna().sum()

0

## Tiền xử lý (tách các tập)
    Ta tách các tập từ dữ liệu thu thập được thành 3 tập: train, validation và test theo tỉ lệ 70%:15%:15%

In [12]:
# Tách X và y
y_sr = data['Giá bán'] # sr là viết tắt của series
X_df = data.drop("Giá bán", axis=1)

In [13]:
# Tách theo tỉ lệ: 7:2:1
# Tách dữ liệu thu thập thành 2 tập: tập huấn luyện và tập other (validation và test) theo tỉ lệ 60%:40%
train_X_df, other_X_df, train_y_sr, other_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)

# Tách tập other (validation và test) thành 2 tập: tập validation và tập test theo tỉ lệ 50%:50% (tức: bằng 15%:15% so với dữ liệu ban đầu)
val_X_df, test_X_df, val_y_sr, test_y_sr = train_test_split(other_X_df, other_y_sr, test_size=0.5, random_state=0)

train_X_df.shape, val_X_df.shape, test_X_df.shape, train_y_sr.shape, val_y_sr.shape, test_y_sr.shape

((4579, 122), (981, 122), (982, 122), (4579,), (981,), (982,))

## Khám phá dữ liệu (tập huấn luyện)

In [14]:
train_X_df.dtypes

Loại hình             object
Phòng ngủ            float64
Phòng tắm            float64
Diện tích             object
Diện tích sử dụng     object
                      ...   
Căn hộ view sông      object
Đầy đủ vệ sinh        object
Kệ bếp                object
Hoàn thiện cơ bản     object
Nhà trống             object
Length: 122, dtype: object

In [15]:
train_X_df.dtypes[train_X_df.dtypes != object]

Phòng ngủ    float64
Phòng tắm    float64
Số tầng      float64
dtype: object

In [16]:
dropped_col = []

In [17]:


# Các cột dạng số
num_cols = ['Phòng ngủ', 'Phòng tắm', 'Số tầng']
df = train_X_df[num_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,Phòng ngủ,Phòng tắm,Số tầng
missing_ratio,1.9,2.3,47.2
min,0.0,1.0,1.0
lower_quartile,2.0,1.0,3.0
median,2.0,2.0,9.0
upper_quartile,3.0,2.0,17.0
max,25.0,25.0,44.0


In [18]:
pd.set_option('display.max_colwidth', 200) # Để nhìn rõ hơn

cat_cols = list(set(train_X_df.columns) - set(num_cols))
df = train_X_df[cat_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
df.agg([missing_ratio, num_values, value_ratios])

for i in range(0, df.shape[1]):
    if df.agg(missing_ratio)[i] >= 50:
         print(df.columns.values[i], df.agg(missing_ratio)[i])
        dropped_col.append(df.columns.values[i])
len(dropped_col)

59

In [19]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_titles=1):
        self.num_top_titles = num_top_titles
    def fit(self, X_df, y=None):
        title_col = X_df.Name.str.extract(r'([a-zA-z]+)\.', expand=False)
        self.title_counts_ = title_col.value_counts()
        titles = list(self.title_counts_.index)
        self.top_titles_ = titles[:max(1, min(self.num_top_titles, len(titles)))]
        return self
    def transform(self, X_df, y=None):
        # YOUR CODE HERE
        df = X_df.copy()
        title_col = df.Name.str.extract("([a-zA-z]+)\.", expand=False)
        title_col[~title_col.isin(self.top_titles_)] = 'Others'
        df["Title"] = title_col
        dropped_cols = ['Name', 'Ticket', 'Cabin']
        df.drop(dropped_cols, axis=1, inplace=True)
        return df